#  Consignes

## Description

Ouvrir le fichier ks-projects-201801.csv, il recense environ 100 000 projets KickStarter. Intégrer les données directement avec L'API Python dans une base de données Mongo. 

Il conviendra de bien spécifier manuellement l'ID du document. Pensez aussi à bien formatter le type des données pour profiter des méthodes implémentées par Mongo. L'ensemble de données n'est pas forcément nécessaire, c'est à vous de créer votre modèle de données.

## Questions

- 1) Récupérer les 5 projets ayant reçu le plus de promesse de dons.
- 2) Compter le nombre de projets ayant atteint leur but.
- 3) Compter le nombre de projets pour chaque catégorie.
- 4) Compter le nombre de projets français ayant été instanciés avant 2016.
- 5) Récupérer les projets américains ayant demandé plus de 200 000 dollars.
- 6) Compter le nombre de projet ayant "Sport" dans leur nom

In [51]:
import pandas as pd
import pymongo

In [52]:
client = pymongo.MongoClient('mongo')
database = client['exercices']
collection = database['kickstarter']

In [53]:
df_ks = pd.read_csv("./data/ks-projects-201801-sample.csv")
df_ks.head()

/root/.local/share/virtualenvs/code-AFz48Hjb/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (6,8,10,12) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real
0,872782264,"Scott Cooper's Solo CD ""A Leg Trick"" (Canceled)",Rock,Music,USD,2011-09-16,2000,2011-08-17 06:31:31,1145,canceled,24,US,1145,1145.000000
1,1326492673,Ohceola jewelry,Fashion,Fashion,USD,2012-08-22,18000,2012-07-23 20:46:48,1851,failed,28,US,1851,1851.000000
2,1688410639,Sluff Off & Harald: Two latest EGGs are Classi...,Tabletop Games,Games,USD,2016-07-19,2000,2016-07-01 21:55:54,7534,successful,254,US,3796,7534.000000
3,156812982,SketchPlanner: Create and Plan- all in one bea...,Art Books,Publishing,USD,2017-09-27,13000,2017-08-28 15:47:02,16298,successful,367,US,2670,16298.000000
4,1835968190,Proven sales with custom motorcycle accessories,Sculpture,Art,CAD,2016-02-24,5000,2016-01-25 17:37:10,1,failed,1,CA,0.708148,0.738225


Ce warning intervient lorsque pandas n'arrive pas à inférer le type de données. Il est sympa il précise les colones 6,8,10,12. 

In [54]:
df_ks.columns[[6,8,10,12]]

Index(['goal', 'pledged', 'backers', 'usd pledged'], dtype='object')

## Question 0

### Netoyer les données

In [55]:
df_ks['launched']= pd.to_datetime(df_ks['launched'], errors = 'coerce')
df_ks = df_ks.dropna(subset = ['launched'])

In [56]:
df_ks = df_ks.rename(columns = {"ID": "_id"})
df_ks

,_id,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real
0,872782264,"Scott Cooper's Solo CD ""A Leg Trick"" (Canceled)",Rock,Music,USD,2011-09-16,2000,2011-08-17 06:31:31,1145,canceled,24,US,1145,1145.000000
1,1326492673,Ohceola jewelry,Fashion,Fashion,USD,2012-08-22,18000,2012-07-23 20:46:48,1851,failed,28,US,1851,1851.000000
2,1688410639,Sluff Off & Harald: Two latest EGGs are Classi...,Tabletop Games,Games,USD,2016-07-19,2000,2016-07-01 21:55:54,7534,successful,254,US,3796,7534.000000
3,156812982,SketchPlanner: Create and Plan- all in one bea...,Art Books,Publishing,USD,2017-09-27,13000,2017-08-28 15:47:02,16298,successful,367,US,2670,16298.000000
4,1835968190,Proven sales with custom motorcycle accessories,Sculpture,Art,CAD,2016-02-24,5000,2016-01-25 17:37:10,1,failed,1,CA,0.708148,0.738225
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149995,574841011,Scutes Headphones - A new way to listen to you...,Product Design,Design,USD,2014-11-14,40000,2014-10-14 20:44:37,3664,canceled,43,US,3664,3664.000000
149996,235190237,My Drunk @$$ Reviews,Webseries,Film & Video,USD,2012-05-18,1000,2012-04-18 04:42:47,0,failed,0,US,0,0.000000
149997,758540498,Freedom Planet - High Speed Platform Game,Video Games,Games,USD,2013-02-15,2000,2013-01-16 00:53:32,25472.7,successful,1105,US,25472.7,25472.690000
149998,1624952469,Clown vs Mime,Webseries,Film & Video,USD,2011-05-27,1000,2011-04-27 01:22:52,76,failed,3,US,76,76.000000


### Importer les données

In [57]:
documents = df_ks.to_dict(orient = 'records')

In [58]:
collection.insert_many(documents)

In [59]:
collection.find_one({})

{'_id': 872782264,
 'name': 'Scott Cooper\'s Solo CD "A Leg Trick" (Canceled)',
 'category': 'Rock',
 'main_category': 'Music',
 'currency': 'USD',
 'deadline': '2011-09-16',
 'goal': 2000.0,
 'launched': datetime.datetime(2011, 8, 17, 6, 31, 31),
 'pledged': 1145.0,
 'state': 'canceled',
 'backers': 24,
 'country': 'US',
 'usd pledged': 1145.0,
 'usd_pledged_real': 1145.0}

## Question 1  

In [60]:
dons = collection.find().sort([("usd_pledged_real", -1)]).limit(5)
list(dons)

[{'_id': 342886736,
  'name': "COOLEST COOLER: 21st Century Cooler that's Actually Cooler",
  'category': 'Product Design',
  'main_category': 'Design',
  'currency': 'USD',
  'deadline': '2014-08-30',
  'goal': '50000.0',
  'launched': datetime.datetime(2014, 7, 8, 10, 14, 37),
  'pledged': '13285226.36',
  'state': 'successful',
  'backers': '62642',
  'country': 'US',
  'usd pledged': '13285226.36',
  'usd_pledged_real': 13285226.36},
 {'_id': 2103598555,
  'name': 'Pebble 2, Time 2 + All-New Pebble Core',
  'category': 'Product Design',
  'main_category': 'Design',
  'currency': 'USD',
  'deadline': '2016-06-30',
  'goal': '1000000.0',
  'launched': datetime.datetime(2016, 5, 24, 15, 49, 52),
  'pledged': '12779843.49',
  'state': 'successful',
  'backers': '66673',
  'country': 'US',
  'usd pledged': '12779843.49',
  'usd_pledged_real': 12779843.49},
 {'_id': 1033978702,
  'name': 'OUYA: A New Kind of Video Game Console',
  'category': 'Gaming Hardware',
  'main_category': 'Games'

## Question 2

In [61]:
but = collection.count_documents({'state' : 'successful'})
print("Le nombre de projets ayant atteint leur but est:", but)

Le nombre de projets ayant atteint leur but est: 53040


## Question 3

In [62]:
projet_categ = collection.aggregate([{"$group" : {"_id" : "$category", "somme" : {"$sum" : 1}}}])
list(projet_categ)

[{'_id': 'Woodworking', 'somme': 433},
 {'_id': 'Video Art', 'somme': 65},
 {'_id': 'Theater', 'somme': 2786},
 {'_id': 'Stationery', 'somme': 90},
 {'_id': 'Sculpture', 'somme': 737},
 {'_id': 'Classical Music', 'somme': 1064},
 {'_id': 'Publishing', 'somme': 2332},
 {'_id': 'Childrenswear', 'somme': 192},
 {'_id': 'Movie Theaters', 'somme': 90},
 {'_id': 'Residencies', 'somme': 32},
 {'_id': 'Horror', 'somme': 525},
 {'_id': 'Rock', 'somme': 2707},
 {'_id': 'Art', 'somme': 3358},
 {'_id': 'Photography', 'somme': 2239},
 {'_id': 'Camera Equipment', 'somme': 165},
 {'_id': 'Flight', 'somme': 158},
 {'_id': 'Journalism', 'somme': 672},
 {'_id': 'Puzzles', 'somme': 95},
 {'_id': 'Webseries', 'somme': 2316},
 {'_id': 'Makerspaces', 'somme': 91},
 {'_id': 'Metal', 'somme': 274},
 {'_id': 'Digital Art', 'somme': 524},
 {'_id': 'Knitting', 'somme': 78},
 {'_id': 'Academic', 'somme': 367},
 {'_id': 'Music Videos', 'somme': 299},
 {'_id': 'Wearables', 'somme': 508},
 {'_id': 'Places', 'somme':

## Question 4

In [63]:
import datetime
year = datetime.datetime.strptime("2016-01-01",'%Y-%m-%d')

In [64]:
projet_categ = {"$and": [{"country":"FR", "launched":{"$lte": year}}]}
a = collection.count_documents(projet_categ)
print("Le nombre de projets francais ayant été instanciés avant 2016 est de :", a)

Le nombre de projets francais ayant été instanciés avant 2016 est de : 330


## Question 5

In [65]:
projet_200 = {"$and": [{"country":"US", "usd_pledged_real":{"$gte": 200000}}]}
b = collection.count_documents(projet_200)
print("Le nombre de projets américains ayant demandé plus de 200 000 dollars :", b)

Le nombre de projets américains ayant demandé plus de 200 000 dollars : 658


## Question 6 

In [66]:
collection.create_index([("name", "text")])
c = collection.count_documents( { "$text": { "$search": "Sport" }})
print("Le nombre de projet ayant Sport dans leur nom :", c)

Le nombre de projet ayant Sport dans leur nom : 318
